# Pi Calculation

This notebook is done as a comparison between the performance of simple `python`, `numpy`, `numba` and `cython` for calculating $\pi$ using Monte Carlo.

This is a training exercise just for me to learn the differences and how does it work.

In [1]:
import numpy as np
import random as rn
import matplotlib.pylab as plt

In [2]:
import numba
import cython

print("Numba",numba.__version__)
print("Cython",cython.__version__)
print("Numpy",np.__version__)


Numba 0.49.0
Cython 0.29.17
Numpy 1.18.3


The method will go as follows, 

> Count the proportion of numbers that are inside the unit circle centered on zero out of randomly generated points inside a square from (0,0) to (1,1), this will lead to $\pi/4$

The `python` version of this code should be something as,

In [3]:
def pi_func(N):
    suma=0
    for i in range(N):
        x=rn.random()
        y=rn.random()
        if x**2+y**2<1:
            suma+=1
    return 4.0*suma/N

But this operation can be vectorized using `numpy`

In [4]:
def pi_func_numpy(N):
    x=np.random.random(N)
    y=np.random.random(N)
    return 4.0*(x**2+y**2<1).sum()/N

Let us simply add the `numba` decorator to the `python` for loop based code and using the `fastmath` flag, so that 

In [5]:
@numba.jit
def pi_func_numb(N):
    suma=0
    for i in range(N):
        x=rn.random()
        y=rn.random()
        if x**2+y**2<1:
            suma+=1
    return 4.0*suma/N

@numba.jit(fastmath=True)
def pi_func_numb2(N):
    suma=0
    for i in numba.prange(N):
        x=rn.random()
        y=rn.random()
        if x**2+y**2<1:
            suma+=1
    return 4.0*suma/N


Let us run these codes and test how long they take

In [6]:
N=10000
print("Python")
%timeit pi_func(N)
print("Numpy")
%timeit pi_func_numpy(N)
print("Numba")
%timeit pi_func_numb(N)
print("Numba fastmath")
%timeit pi_func_numb2(N)

Python
4.2 ms ± 47.7 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
Numpy
190 µs ± 3.56 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
Numba
127 µs ± 1.54 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)
Numba fastmath
129 µs ± 1.78 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


Now, let us introduce `Cython`, so to be able to use it on these notebooks, the magic cell option has to be imported

In [7]:
%load_ext Cython

And now let us construct the `cython` function with all the cdefs including the datatypes needed to be as much efficient as possible. (At least as I can think right now.). We are using `-a` flag so that we can see which lines are taking longer.

In [8]:
%%cython -a
cimport cython
from libc.stdlib cimport rand

cdef extern from "limits.h":
    int RAND_MAX    
@cython.cdivision(True)
cpdef double pi_func_cy(int N):
    cdef int suma=0
    cdef int i
    cdef double x
    cdef double y
    cdef double res
    for i in range(N):
        x=rand()
        y=rand()
        if x**2+y**2<RAND_MAX**2:
            suma+=1
    res=4.0*suma/N
    return res   

And let us compare all of the functions with the same amount of numbers,

In [9]:
N=10000000
print("Python")
%timeit pi_func(N)
print("Numpy")
%timeit pi_func_numpy(N)
print("Numba")
%timeit pi_func_numb(N)
print("Numba fastmath")
%timeit pi_func_numb2(N)
print("Cython")
%timeit pi_func_cy(N)

Python
4.24 s ± 39.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
Numpy
218 ms ± 2.12 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
Numba
128 ms ± 829 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
Numba fastmath
120 ms ± 785 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
Cython
96.5 ms ± 335 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


Just in this simple example, the time can be reduced drastically from pure `python` and to the half of `numpy` vectorized version. 